## Configuração de API

In [1]:
CLIENT_ID = "SEUID"
SECRET_KEY = "SUASENHA"

In [1]:
import requests
import pandas as pd

In [3]:
auth = requests.auth.HTTPBasicAuth(CLIENT_ID, SECRET_KEY)

In [4]:
with open('pw.txt', 'r') as f:
    pw = f.read()

In [5]:
data = {
    'grant_type': 'password',
    'username': 'SEUUSER',
    'password': pw
}

In [6]:
headers = {'User-Agent': 'MyAPI/0.01'}

In [7]:
res = requests.post('https://www.reddit.com/api/v1/access_token', 
                   auth=auth, data=data, headers=headers)

In [8]:
TOKEN = res.json()['access_token']

In [9]:
headers = {**headers, **{'Authorization': f'bearer {TOKEN}'}}

In [10]:
# headers

In [11]:
# requests.get('https://oauth.reddit.com/api/v1/me', headers={'User-Agent': 'MyAPI/0.01'})

In [12]:
# requests.get('https://oauth.reddit.com/api/v1/me', headers=headers).json()

In [13]:
res = requests.get('https://oauth.reddit.com/r/popheads//hot',
                headers=headers, params={'limit': '100'})

In [14]:
all_posts_data = []

for post in res.json()['data']['children']:
    post_data_dict = {
        'id_post': post['data'].get('id'),
        'title': post['data'].get('title'),
        'author': post['data'].get('author'),
        'subreddit': post['data'].get('subreddit'),
        'ups': post['data'].get('ups'),
        'upvote_ratio': post['data'].get('upvote_ratio'),
        'downs': post['data'].get('downs'),
        'score': post['data'].get('score'),
        'num_comments': post['data'].get('num_comments'),
        'is_self': post['data'].get('is_self'), 
        'selftext': post['data'].get('selftext', ''),
        'created_utc': post['data'].get('created_utc')
    }
    all_posts_data.append(post_data_dict)

df = pd.DataFrame(all_posts_data)

## Descrição da Base de Dados e Atributos Coletados

### Atributos Originais

Para esta atividade, foram coletados os dados sobre os posts do subreddit "popheads". A coleta foi feita através da API do Reddit, resultando em um conjunto de 102 postagens. 
Esta base de dados, ainda em seu formato bruto, contém as seguintes informações para cada postagem:
- id_post: ID do post;
- title: Título do post;
- author: Autor do post;
- subreddit: Subreddit referente ao post;
- ups: Número de "upvotes" que o post recebeu;
- upvote_ratio: Proporção de "upvotes" em relação ao total de votos ("upvotes" + "downvotes");
- downs: Número de "downvotes" que o post recebeu;
- score: Pontuação do post calculada como "ups" - "downs";
- num_commets: Número de comentários do post;
- is_self: : Indica se o post é um post de texto (True) ou post de um link externo (False);
- selftext: Contém o corpo do texto do post (caso seja um post de texto);
- created_ufc: Timestamp Unix da criação do post

### Atributos Derivados

Número de caracteres presentes no título da postagem. Essa informação pode ajudar a analisar se a concisão ou a extenão do título influenciam o engajamento ou o tipo de conteúdo.

In [15]:
df['len_title'] = df['title'].str.len()

Representação em formato de data e hora legível do timestamp Unix da criação da postagem. Essa informação é útil pois ajuda a extrair informações temporais mais granulares.

In [16]:
df['datetime'] = pd.to_datetime(df['created_utc'], unit='s', utc=True)

A hora do dia em que a postagem foi criada. Essa informação auxilia a identificar os horários de pico de atividade na comunidade.

In [17]:
df['post_hour'] = df['datetime'].dt.hour

O dia da semana (0 para segunda-feira a 6 para domingo) em que a postagem foi criada. Com essa informação, é possível analisar padrões de postagem e engajamento ao longo da semana.

In [18]:
df['post_day_of_week'] = df['datetime'].dt.dayofweek

Representa engajamento total da postagem, calculada como a soma de votos positivos e número de comentários. Isso oferece uma visão consolidada da popularidade e da capacidade de discussão que a postagem gerou.

In [42]:
df['engagement_score'] = df['ups'] + df['num_comments']

## Dataframe

In [24]:
import pandas as pd

In [139]:
df = pd.read_csv('df.csv')

## Análise de sentimento da postagem

### Pré-processamento

#### Normalização do título e do texto da postagem

In [140]:
import numpy as np
import nltk

In [141]:
df['selftext'] = df['selftext'].replace('NaN', np.nan)
df['selftext'] = df['selftext'].replace('', np.nan)

df['title'] = df['title'].replace('NaN', np.nan)
df['title'] = df['title'].replace('', np.nan)

df['title'] = df['title'].str.lower()
df['selftext'] = df['selftext'].str.lower()

df_text_processing = df[['id_post', 'title', 'selftext']].copy()

In [142]:
df_text_processing['selftext'] = df_text_processing['selftext'].astype(str).fillna('')
df_text_processing['title'] = df_text_processing['title'].astype(str).fillna('')

#### Conversão de emojis

In [143]:
import emoji 
import re 

In [144]:
def convert_unicode(text):
    return emoji.demojize(text, language='en', delimiters=(" ", " "))

df_text_processing['title'] = df_text_processing['title'].apply(convert_unicode)
df_text_processing['selftext'] = df_text_processing['selftext'].apply(convert_unicode)

emoticon_mapping = {
    r':\)\s*': ' _happy_face_ ',  # :)
    r':-\)\s*': ' _happy_face_ ', # :-)
    r'\(\s*:\)': ' _happy_face_ ', # (:
    r':D\s*': ' _big_smile_ ',   # :D
    r':\(\s*': ' _sad_face_ ',   # :(
    r':-\(\s*': ' _sad_face_ ',  # :-(
    r';\)\s*': ' _winking_face_ ', # ;)
    r':\/\s*': ' _confused_face_ ', # :/
    r':O\s*': ' _surprise_face_ ', # :O
    r'xD\s*': ' _laughing_face_ ', # xD
}

def convert_emoticons(text):
    for pattern, replacement in emoticon_mapping.items():
        text = re.sub(pattern, replacement, text, flags=re.IGNORECASE)
    return text

df_text_processing['title'] = df_text_processing['title'].apply(convert_emoticons)
df_text_processing['selftext'] = df_text_processing['selftext'].apply(convert_emoticons)

#### Tokenização

In [145]:
from nltk import word_tokenize
from nltk.tokenize import RegexpTokenizer

In [146]:
tokenizer = RegexpTokenizer(r'\w+')

df_text_processing['title_tokens'] = df_text_processing['title'].apply(tokenizer.tokenize)
df_text_processing['selftext_tokens'] = df_text_processing['selftext'].apply(tokenizer.tokenize)

#### Remoção de stop words

In [147]:
from nltk.corpus import stopwords

In [148]:
stop_words = set(stopwords.words('english'))

def remove_stopwords(tokens):
    return [word for word in tokens if word.lower() not in stop_words]

df_text_processing['title_stopwords'] = df_text_processing['title_tokens'].apply(remove_stopwords)
df_text_processing['selftext_stopwords'] = df_text_processing['selftext_tokens'].apply(remove_stopwords)

#### Lematização

In [149]:
from nltk.stem import WordNetLemmatizer

In [150]:
lemmatizer = WordNetLemmatizer()

def lemmatize_tokens_cleaned(tokens):
    return [lemmatizer.lemmatize(word, pos='n') for word in tokens]

df_text_processing['title_lemmas'] = df_text_processing['title_stopwords'].apply(lemmatize_tokens_cleaned)
df_text_processing['selftext_lemmas'] = df_text_processing['selftext_stopwords'].apply(lemmatize_tokens_cleaned)

### TF-IDF

In [151]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [152]:
df_text_processing['combined_lemmas'] = df_text_processing.apply(
    lambda row: row['title_lemmas'] + row['selftext_lemmas'], axis=1
)

df_text_processing['text_for_tfidf'] = df_text_processing['combined_lemmas'].apply(lambda tokens: ' '.join(tokens))

#### Removendo números após a junção dos textos

In [153]:
def remove(text):
    if pd.isna(text):
        return text
    result = re.sub(r'\d+', '', str(text)) 
    return result.strip()

df_text_processing['text_for_tfidf'] = df_text_processing['text_for_tfidf'].apply(remove)

In [162]:
df_text_processing

,id_post,title,selftext,title_tokens,selftext_tokens,title_stopwords,selftext_stopwords,title_lemmas,selftext_lemmas,combined_lemmas,text_for_tfidf
0,1lihdmp,"daily discussion - june 23, 2025","talk about anything, music related or not. how...","[daily, discussion, june, 23, 2025]","[talk, about, anything, music, related, or, no...","[daily, discussion, june, 23, 2025]","[talk, anything, music, related, however, pop,...","[daily, discussion, june, 23, 2025]","[talk, anything, music, related, however, pop,...","[daily, discussion, june, 23, 2025, talk, anyt...",daily discussion june talk anything music re...
1,1lixuol,"the popheads charts, june 23rd, 2025: a top te...",[hot 50 spotify playlist](https _confused_face...,"[the, popheads, charts, june, 23rd, 2025, a, t...","[hot, 50, spotify, playlist, https, _confused_...","[popheads, charts, june, 23rd, 2025, top, ten,...","[hot, 50, spotify, playlist, https, _confused_...","[popheads, chart, june, 23rd, 2025, top, ten, ...","[hot, 50, spotify, playlist, http, _confused_f...","[popheads, chart, june, 23rd, 2025, top, ten, ...",popheads chart june rd top ten stale billboar...
2,1lj1i2x,pitchfork review: benson boone - american hear...,nan,"[pitchfork, review, benson, boone, american, h...",[nan],"[pitchfork, review, benson, boone, american, h...",[nan],"[pitchfork, review, benson, boone, american, h...",[nan],"[pitchfork, review, benson, boone, american, h...",pitchfork review benson boone american heart ...
3,1lip81p,"cardi b announces new album ""am i the drama?"",...",nan,"[cardi, b, announces, new, album, am, i, the, ...",[nan],"[cardi, b, announces, new, album, drama, septe...",[nan],"[cardi, b, announces, new, album, drama, septe...",[nan],"[cardi, b, announces, new, album, drama, septe...",cardi b announces new album drama september nan
4,1lj0y7x,benson boone - american heart (theneedledrop a...,nan,"[benson, boone, american, heart, theneedledrop...",[nan],"[benson, boone, american, heart, theneedledrop...",[nan],"[benson, boone, american, heart, theneedledrop...",[nan],"[benson, boone, american, heart, theneedledrop...",benson boone american heart theneedledrop albu...
...,...,...,...,...,...,...,...,...,...,...,...
97,1lgbnks,the knocks &amp; dragonette - friday night,nan,"[the, knocks, amp, dragonette, friday, night]",[nan],"[knocks, amp, dragonette, friday, night]",[nan],"[knock, amp, dragonette, friday, night]",[nan],"[knock, amp, dragonette, friday, night, nan]",knock amp dragonette friday night nan
98,1lgeyf0,audrey hobert - bowling alley (official video),nan,"[audrey, hobert, bowling, alley, official, video]",[nan],"[audrey, hobert, bowling, alley, official, video]",[nan],"[audrey, hobert, bowling, alley, official, video]",[nan],"[audrey, hobert, bowling, alley, official, vid...",audrey hobert bowling alley official video nan
99,1lfujc5,katseye - gabriela,nan,"[katseye, gabriela]",[nan],"[katseye, gabriela]",[nan],"[katseye, gabriela]",[nan],"[katseye, gabriela, nan]",katseye gabriela nan
100,1lgcz4q,mura masa &amp; george riley - forever,nan,"[mura, masa, amp, george, riley, forever]",[nan],"[mura, masa, amp, george, riley, forever]",[nan],"[mura, masa, amp, george, riley, forever]",[nan],"[mura, masa, amp, george, riley, forever, nan]",mura masa amp george riley forever nan


#### Aplicação do algoritmo

In [154]:
vectorizer = TfidfVectorizer()

tfidf_matrix = vectorizer.fit_transform(df_text_processing['text_for_tfidf'])

tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())

In [155]:
c = tfidf_df.columns[ (tfidf_df > 0).any() ].tolist()

In [156]:
c

['_confused_face_',
 '_happy_face_',
 '_sad_face_',
 '_ultimate_rate_ariana_grande_vs_beyonce_vs',
 'aaron',
 'abdff',
 'abrams',
 'abrupt',
 'acceptable',
 'accepted',
 'acclaimed',
 'accompanied',
 'accomplishment',
 'accusation',
 'achievement',
 'across',
 'act',
 'active',
 'activity',
 'actual',
 'actuality',
 'actually',
 'ada',
 'add',
 'addison',
 'adored',
 'adult',
 'aftercare',
 'agent',
 'ago',
 'ahead',
 'aimed',
 'ala',
 'album',
 'alesha',
 'alex',
 'alien',
 'align',
 'all__bts_members_are_now_done_with_mandatory',
 'alley',
 'alliigator',
 'allowed',
 'almost',
 'along',
 'alongside',
 'already',
 'also',
 'alter',
 'although',
 'always',
 'amaarae',
 'amazing',
 'amen',
 'america',
 'american',
 'americana',
 'ameriican',
 'amid',
 'among',
 'amongst',
 'amonthly',
 'amor',
 'amp',
 'anniversary',
 'announced',
 'announcement',
 'announces',
 'another',
 'answer',
 'anthology',
 'anticipated',
 'anxiety',
 'anybagel',
 'anything',
 'aobeats',
 'apnews',
 'appears',
 

### Modelagem de tópicos com NMF


In [157]:
from sklearn.decomposition import NMF


#### Aplicação do algoritmo

In [158]:
model = NMF(n_components=10, random_state=0)
model.fit(tfidf_matrix)

,n_components,10
,init,None
,solver,'cd'
,beta_loss,'frobenius'
,tol,0.0001
,max_iter,200
,random_state,0
,alpha_W,0.0
,alpha_H,'same'
,l1_ratio,0.0
,verbose,0


#### DataFrame com os tópicos e as palavras associadas

In [159]:
num_palavras = 10

dicionario_topicos = {}

for indice_topico, topico in enumerate(model.components_):
    
    chave_topico = f"Tópico {indice_topico}"
    
    feature_names = vectorizer.get_feature_names_out()
    dicionario_topicos[chave_topico] = [feature_names[i] for i in topico.argsort()[:-num_palavras - 1:-1]]

df_topicos = pd.DataFrame(dicionario_topicos)


In [160]:
df_topicos


,Tópico 0,Tópico 1,Tópico 2,Tópico 3,Tópico 4,Tópico 5,Tópico 6,Tópico 7,Tópico 8,Tópico 9
0,_confused_face_,gossip,new,boone,katseye,haim,leg,lorde,beach,warren
1,http,content,album,benson,gabriela,nan,mccall,hammer,caroline,alex
2,popheads,news,announces,american,nan,live,wet,nan,polachek,ordinary
3,com,allowed,night,heart,chaos,lounge,davina,pitchfork,stranding,week
4,www,banned,nan,review,medley,wrong,video,track,nan,hot
5,reddit,removed,single,pitchfork,highlight,addison,official,review,death,billboard
6,amp,link,maroon,theneedledrop,choice,rae,nan,charli,lyric,top
7,rate,post,friday,album,gnarly,rampage,twilight,remix,video,add
8,comment,music,tour,fun,performs,set,zone,xcx,moyka,third
9,th,without,release,half,kid,headphone,audrey,year,deep,chart


### Colocando label nos tópicos

In [166]:


W_matrix = model.transform(tfidf_matrix)

df_text_processing['topic'] = np.argmax(W_matrix, axis=1)

mapa_topicos = {
    0: "Meta-Discussão e Links",
    1: "Regras e Moderação",
    2: "Lançamentos de Álbuns/Singles",
    3: "Críticas e Reviews de Álbuns",
    4: "Performances Ao Vivo ",
    5: "Performances e Sessões ",
    6: "Artistas e Clipes Específicos ",
    7: "Discussões de Artistas",
    8: "Discussões de Artistas",
    9: "Desempenho em Charts e Billboard"
}

df_text_processing['topic_label'] = df_text_processing['topic'].map(mapa_topicos)

